## **Urdu Speech Emotion Classification**

[Click here to read blog ](https://medium.com/@rameeshamalik.143/building-an-urdu-speech-emotion-detection-with-deepgram-transformers-71f0a7f2e216)

# Step 1: Install Required Libraries


1.   **Deepgram SDK:** Converts speech to text.
2.   **Transformers:** Provides pre-trained models for text classification.
3.   **Datasets:** Helps manage and preprocess text data.
4.   **Streamlit:** Makes it easy to build web apps.
5. **Pyngrok:** Deploys the app online.

# Step 2: Load Your Dataset

# Step 3: Prepare the Dataset

# Step 4: Transcribe Speech with Deepgram

**Setup Deepgram**

**Transcribe Audio Files**

# Step 5: Fine-Tune a Model for Emotion Detection

**Preprocess Data**

**Train the Model**

# Step 6: Build the Web App

# Step 7: Run the App

# **New Section**

In [ ]:
!pip install deepgram-sdk transformers datasets
!pip install sentencepiece sacremoses


In [ ]:
from deepgram import DeepgramClient  # Import DeepgramClient
import requests
import json

# Initialize Deepgram API key
DEEPGRAM_API_KEY = "4f6f3f1686dd74ab520e485fb1edaf651750999e"
deepgram = DeepgramClient(api_key=DEEPGRAM_API_KEY)  # Use DeepgramClient instead of Deepgram

# Function to transcribe audio
def transcribe_audio(audio_file_path):
    with open(audio_file_path, "rb") as audio:
        response = requests.post(
            "https://api.deepgram.com/v1/listen",
            headers={
                "Authorization": f"Token {DEEPGRAM_API_KEY}",
                "Content-Type": "audio/wav",  # Ensure your file is in WAV format
            },
            data=audio,
        )
        result = response.json()
        return result["results"]["channels"][0]["alternatives"][0]["transcript"]

# Upload audio file
from google.colab import files
uploaded = files.upload()  # Upload an audio file
audio_file_path = list(uploaded.keys())[0]

# Transcribe the uploaded audio file
try:
    transcript = transcribe_audio(audio_file_path)
    print("Transcription:", transcript)
except Exception as e:
    print("An error occurred during transcription:", str(e))

Saving SM27_F1_S093.wav to SM27_F1_S093.wav
Transcription: 


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load translation model (Urdu to English)
ur_to_en_model_name = "Helsinki-NLP/opus-mt-ur-en"
ur_to_en_tokenizer = MarianTokenizer.from_pretrained(ur_to_en_model_name)
ur_to_en_model = MarianMTModel.from_pretrained(ur_to_en_model_name)

# Load translation model (English to Urdu)
en_to_ur_model_name = "Helsinki-NLP/opus-mt-en-ur"
en_to_ur_tokenizer = MarianTokenizer.from_pretrained(en_to_ur_model_name)
en_to_ur_model = MarianMTModel.from_pretrained(en_to_ur_model_name)

def back_translate(text):
    # Urdu to English
    en_tokens = ur_to_en_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    en_translation = ur_to_en_model.generate(**en_tokens)
    en_text = ur_to_en_tokenizer.decode(en_translation[0], skip_special_tokens=True)

    # English to Urdu
    ur_tokens = en_to_ur_tokenizer(en_text, return_tensors="pt", padding=True, truncation=True)
    ur_translation = en_to_ur_model.generate(**ur_tokens)
    ur_text = en_to_ur_tokenizer.decode(ur_translation[0], skip_special_tokens=True)

    return ur_text

# Example usage
new_text = back_translate(transcript)
print("Back Translated Text:", new_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

Back Translated Text: صرف وہی لوگ خدا کے رحم سے مستفید ہوتے ہیں ۔


In [ ]:
# Save back-translated sentences to a file
with open("new_corpus.txt", "w", encoding="utf-8") as file:
    file.write(new_text)


In [ ]:
from datasets import Dataset
from transformers import GPT2Tokenizer

# Create dataset
data = {"text": [new_text]}
dataset = Dataset.from_dict(data)

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer.pad_token = tokenizer.eos_token

# Tokenize dataset
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPTNeoForCausalLM, Trainer, TrainingArguments

# Load a smaller model for less memory usage
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

# Training arguments with optimizations
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="no",  # Disable evaluation
    learning_rate=5e-5,
    weight_decay=0.01,
    per_device_train_batch_size=1,  # Reduce batch size
    num_train_epochs=1,
    gradient_accumulation_steps=16,  # Accumulate gradients
    fp16=True,  # Mixed precision
)

# Enable gradient checkpointing
model.gradient_checkpointing_enable()

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset  # Ensure this is tokenized and of manageable size
)

# Fine-tune the model
trainer.train()


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: f248010 (f248010-national-university-of-computer-and-emerging-sci). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
from transformers import GPTNeoForCausalLM, Trainer, TrainingArguments

# Load a smaller model for less memory usage
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

# Training arguments with optimizations
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="no",  # Disable evaluation
    learning_rate=5e-5,
    weight_decay=0.01,
    per_device_train_batch_size=1,  # Reduce batch size
    num_train_epochs=1,
    gradient_accumulation_steps=16,  # Accumulate gradients
    fp16=True,  # Mixed precision
)

# Enable gradient checkpointing
model.gradient_checkpointing_enable()


def tokenize_function(example):
    # Tokenize the text and create labels
    tokenized = tokenizer(example["text"], padding="max_length", truncation=True)
    # The labels are the same as the input_ids shifted to the right by one position.
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset  # Ensure this is tokenized and of manageable size
)

# Fine-tune the model
trainer.train()

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=1, training_loss=0.49137985706329346, metrics={'train_runtime': 136.878, 'train_samples_per_second': 0.007, 'train_steps_per_second': 0.007, 'total_flos': 1044828389376.0, 'train_loss': 0.49137985706329346, 'epoch': 1.0})

In [ ]:
!pip install textaugment
from textaugment import EDA

aug = EDA()
augmented_text = aug.synonym_replacement(new_text)
print("Augmented Text:", augmented_text)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15719 sha256=d57d1ce94f2bed907c0d0cd2b65df7107c4a2af371c5c0f3272f00255efaa7a5
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Success

AttributeError: module 'httpcore' has no attribute 'SyncHTTPTransport'

In [ ]:
!pip install nlpaug

import nlpaug.augmenter.word as naw

def augment_text(text):
    aug = naw.SynonymAug(aug_src='wordnet')
    return aug.augment(text)

# Example
augmented_text = augment_text("آپ کیسے ہیں؟")
print("Augmented:", augmented_text)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 5.5 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
!pip install nlpaug
import nltk

# Download the required NLTK resource
nltk.download('averaged_perceptron_tagger')

import nlpaug.augmenter.word as naw

def augment_text(text):
    aug = naw.SynonymAug(aug_src='wordnet')
    return aug.augment(text)

# Example
augmented_text = augment_text("آپ کیسے ہیں؟")
print("Augmented:", augmented_text)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# prompt: Step 6: Pre- and Post-Augmentation Classification for above code

# Pre-augmentation classification (example using a placeholder classifier)
def classify_text(text):
    # Replace this with your actual classification logic
    # This is a placeholder that returns a random classification
    import random
    emotions = ["happy", "sad", "angry", "neutral"]
    return random.choice(emotions)

pre_aug_classification = classify_text(new_text)
print("Pre-augmentation classification:", pre_aug_classification)


# Post-augmentation classification (example)
post_aug_classification = classify_text(augmented_text)
print("Post-augmentation classification:", post_aug_classification)

Pre-augmentation classification: angry
Post-augmentation classification: sad


In [ ]:
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
import nlpaug.augmenter.word as naw

# Download the required NLTK resources
nltk.download('averaged_perceptron_tagger_eng') # Download the correct resource
nltk.download('punkt')  # Add this to ensure tokenization works properly

def augment_text(text):
    aug = naw.SynonymAug(aug_src='wordnet')
    return aug.augment(text)


augmented_text = augment_text("آپ کیسے ہیں؟")
print("Augmented:", augmented_text)

Augmented: ['آپ کیسے ہیں ؟']


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
